In [1]:
%load_ext tensorboard

In [2]:
import tensorflow as tf
import datetime

In [3]:
!rm -rf ./logs/ 

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])


In [5]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])


Epoch 1/5
1875/1875 [==============================] - 28s 15ms/step - loss: 0.2209 - accuracy: 0.9344 - val_loss: 0.0978 - val_accuracy: 0.9700
Epoch 2/5
1875/1875 [==============================] - 22s 12ms/step - loss: 0.0953 - accuracy: 0.9717 - val_loss: 0.0726 - val_accuracy: 0.9772
Epoch 3/5
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0684 - accuracy: 0.9790 - val_loss: 0.0685 - val_accuracy: 0.9783
Epoch 4/5
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0542 - accuracy: 0.9826 - val_loss: 0.0705 - val_accuracy: 0.9791
Epoch 5/5
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0435 - accuracy: 0.9860 - val_loss: 0.0735 - val_accuracy: 0.9764


In [6]:
%tensorboard --logdir logs/fit

In [7]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)


In [8]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()


In [9]:
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')


In [10]:
def train_step(model, optimizer, x_train, y_train):
  with tf.GradientTape() as tape:
    predictions = model(x_train, training=True)
    loss = loss_object(y_train, predictions)
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  train_loss(loss)
  train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
  predictions = model(x_test)
  loss = loss_object(y_test, predictions)

  test_loss(loss)
  test_accuracy(y_test, predictions)


In [11]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)


In [12]:
model = create_model() # reset our model

EPOCHS = 5

for epoch in range(EPOCHS):
  for (x_train, y_train) in train_dataset:
    train_step(model, optimizer, x_train, y_train)
  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

  for (x_test, y_test) in test_dataset:
    test_step(model, x_test, y_test)
  with test_summary_writer.as_default():
    tf.summary.scalar('loss', test_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
  
  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(), 
                         train_accuracy.result()*100,
                         test_loss.result(), 
                         test_accuracy.result()*100))

  # Reset metrics every epoch
  train_loss.reset_states()
  test_loss.reset_states()
  train_accuracy.reset_states()
  test_accuracy.reset_states()


Epoch 1, Loss: 0.2433614730834961, Accuracy: 92.83333587646484, Test Loss: 0.10864152014255524, Test Accuracy: 96.77000427246094
Epoch 2, Loss: 0.10299104452133179, Accuracy: 96.93499755859375, Test Loss: 0.09233210235834122, Test Accuracy: 97.0999984741211
Epoch 3, Loss: 0.07234489917755127, Accuracy: 97.77999877929688, Test Loss: 0.06706105917692184, Test Accuracy: 97.93000030517578
Epoch 4, Loss: 0.0533229298889637, Accuracy: 98.37333679199219, Test Loss: 0.06658702343702316, Test Accuracy: 98.07999420166016
Epoch 5, Loss: 0.04371969774365425, Accuracy: 98.62166595458984, Test Loss: 0.06413280218839645, Test Accuracy: 97.8699951171875


In [13]:
%tensorboard --logdir logs/gradient_tape